In [5]:
import sys
import os
os.chdir("../../")

print(sys.version)

3.8.2 (default, Mar 25 2020, 17:03:02) 
[GCC 7.3.0]


In [6]:
import slim

In [86]:
"""
A script that downloads and parses Salmon Scotland mortality reports.
"""

from bs4 import BeautifulSoup
from requests import get
import os
from pathlib import Path
import tabula

WEBSITE = "https://www.salmonscotland.co.uk"
REPORT_URL = f"{WEBSITE}/reports/monthly-mortality-rate-%s-%d"


async def download(month: str, year: int):
    url = REPORT_URL % (month, year)
    parse_page = get(url).content
    parser = BeautifulSoup(parse_page, "html.parser")
    div = parser.find("div", class_="download-link")
    a = div.find("a")
    download_link = WEBSITE + a["href"]
    return download_link

    """
    report_out_folder = Path("output/reports/")
    filename = report_out_folder / f"SS-{month}-{year}.pdf"
    os.makedirs(str(report_out_folder), exist_ok=True)
    downloaded_pdf = get(download_link)

    with filename.open("wb") as f:
        f.write(downloaded_pdf.content)

    return filename
    """
# download("November", 2021)

# TODO: add tabula execution and parsing...

In [87]:
import tabula

In [91]:
import pandas as pd

pdf = "output/reports/SS-November-2021.pdf"

column_names = ["company", "site", "mortality", "note", "cumulative_mortality"]

complete_df = None
def get_pdf_page(pdf, page, month, year):
    # parsing of multiple pages breaks
    df = tabula.read_pdf(pdf, pages=page, pandas_options={"names": column_names})[0]
    if page == 3:
        df = df[3:].reset_index(drop=True)
    df["month"] = month
    df["year"] = year
    return df

async def get_pdf_pages(pdf, month, year):
    dfs = [get_pdf(page, month, year) for page in range(3, 7)]
    return pd.concat(dfs, ignore_index=True, sort=False)

In [92]:
# extract all 2020...
import asyncio
import datetime

def extract_year_report(year):
    async def parse_and_extract(month):
        print(f"Parsing {month}")
        month_label = datetime.date(year, month, 1).strftime('%B')
        location = await download(month_label, year)
        return await get_pdf_pages(location, month, year)
    
    results = asyncio.gather(*map(parse_and_extract, range(1, 13)))
    dfs = await results
    return pd.concat(dfs, ignore_index=True, keys=["company", "month", "year"])

In [ ]:
await extract_year_report(2020)

Parsing 1
Parsing 2
Parsing 3
Parsing 4
Parsing 5
Parsing 6
Parsing 7
Parsing 8
Parsing 9


In [75]:
df = _

array([1, 2, 3])